In [ ]:
import numpy as np
import os
import subprocess
import gzip

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv1D, AveragePooling1D, MaxPooling1D, TimeDistributed, LeakyReLU, BatchNormalization, Flatten
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras import regularizers

import keras
from keras.models import Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate, Conv1D, BatchNormalization, Flatten
from keras.optimizers import Adagrad, Adam
from keras.utils import plot_model
from keras import regularizers

from keras.utils import plot_model
import pickle
from keras.models import load_model

import numpy as np
from keras.models import load_model
from sklearn.model_selection import KFold
from time import time
from timeit import default_timer as timer
import pickle
import gzip



In [ ]:
dataset = np.load('all_dataset_window19MiddleQ2.npy') # load

In [2]:
f = gzip.GzipFile('all_dataset_window19MiddleQ3.npy.gz', "r")
dataset = np.load(f)

In [3]:
print(dataset.shape)

(1139033, 19, 45)


In [ ]:
f = gzip.GzipFile('all_dataset_window19Middle.npy.gz', "w")
np.save(f, dataset)
f.close()

In [5]:
windowSize = 19
predictionIndex = 9
classSize = 3  # 2 or 3 
numberOfFeatures = 50 #44 

amino_acid_residues = 21
num_classes = 3

In [6]:
LR = 0.0005
drop_out = 0.3
batch_dim = 64
nn_epochs = 5
w_reg = regularizers.l2(0.0001)
number_filters = 16


def get_model():
   
    LR = 0.0005
    drop_out = 0.3
    batch_dim = 64
    nn_epochs = 5
    w_reg = regularizers.l2(0.0001)
    number_filters = 16

    loss = 'categorical_crossentropy'


    input_shape = (windowSize, 21)

    conv1_input = Input(shape=(windowSize, 21), name='InputWindow')

    conv_1 = Conv1D( 64, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg)(conv1_input)
    conv_1 = BatchNormalization(name='BN1')(conv_1)
    conv_2 = Conv1D( 64, 3, strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_1)
    conv_2 = BatchNormalization(name='BN2')(conv_2)
    conv_3 = Conv1D( 64, 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_2)
    conv_3 = BatchNormalization(name='BN3')(conv_3)



    conv_4 = Conv1D( 64, 3, strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg)(conv_3)
    conv_4 = BatchNormalization(name='BN4')(conv_4)
    conv_5 = Conv1D( 64, 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_4)
    conv_5 = BatchNormalization(name='BN5')(conv_5)
    #conv_6 = Conv1D( 73, 3, strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_5)
    #conv_6 = BatchNormalization(name='BN6')(conv_6)


    #conv_7 = Conv1D( 73, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg)(conv_6)
    #conv_7 = BatchNormalization(name='BN7')(conv_7)
    #conv_8 = Conv1D( 4, 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_7)
    #conv_8 = BatchNormalization(name='BN8')(conv_8)
    #conv_9 = Conv1D( 4, 3,  strides=1, padding='same', activation='relu', use_bias=True,kernel_regularizer=w_reg)(conv_8)
    #conv_9 = BatchNormalization(name='BN9')(conv_9)


    flatten  = Flatten()(conv_5)
    first_dense = Dense(16, activation='relu', use_bias=True,  kernel_regularizer=w_reg, name='last')(flatten)
    first_dense = BatchNormalization(name='BN10')(first_dense)
    final_model_output = Dense(num_classes, activation = 'softmax', name='softmax')(first_dense)

    m = Model(inputs=conv1_input, outputs=final_model_output)

    opt = Adam(lr=LR)
    m.compile(optimizer=opt, loss=loss,metrics=['accuracy', 'mae'])

    print("\nHyper Parameters\n")
    print("Learning Rate: " + str(LR))
    print("Drop out: " + str(drop_out))
    print("Batch dim: " + str(batch_dim))
    print("Number of epochs: " + str(nn_epochs))
    print("Regularizers: " + str(w_reg.l2))
    print("\nLoss: " + loss + "\n")
    m.summary()

    #import os
    #os.environ["PATH"] += os.pathsep + 'C:/Users/Ieremie/Anaconda3/pkgs/graphviz-2.38-hfd603c8_2/Library/bin'

    #plot_model(m)#, to_file='model.png')
    return m
    

In [7]:
from sklearn.utils import shuffle
dataset = shuffle(dataset, random_state=111)
print(dataset.shape)
print(dataset[0,:2,:])

(1139033, 19, 45)
[[0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         1.
  0.14804719 0.02003571 0.34298956 0.57444251 0.03732689 0.14063814
  0.99275368 0.18242553 0.82920456 0.18846732 0.08627421 0.96739054
  0.03992533 0.4551211  0.92757356 0.27289176 0.56954622 0.11920292
  0.01781043 0.26894143 0.3078905 ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         1.
  0.08393843 0.01087881 0.06238585 0.35205919 0.01077173 0.04521748
  0.41338244 0.09975048 0.99715686 0.01567329 0.03992533 0.11007258
  0.03954378 0.73885    0.81305736 0.81153268 0.54239792 0.07242649
  0.0090133  0.26894143 0.21416503]]


In [ ]:
kf = KFold(n_splits=10, random_state=10, shuffle=True)
for train, test in kf.split(dataset):
    print(train,test)

In [ ]:
l = 0
r = 113350
n = 1133502

fold = 0
valid_indexes = [()]


for i in range (10):
  
    fold += 1
    print(fold)
    
    m = get_model()
    start_time = timer()
    history = m.fit(np.concatenate((dataset[0:l], dataset[r:n]), axis=0)[:,:,0:21], np.concatenate((dataset[0:l], dataset[r:n]), axis=0)[:, 0, 21 : (21+classSize)],
                    epochs=nn_epochs, validation_data=(dataset[l:r][:,:,0:21], dataset[l:r][:, 0, 21 : (21+classSize)]) ,shuffle=True,batch_size=batch_dim)
    end_time = timer()
    print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")
    
    scores = m.evaluate(dataset[l:r][:,:,0:21], dataset[l:r][:, 0, 21 : (21+classSize)])
    print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]) + ' fold ' + str(fold))
    
    
    m.save('model_kfold' + str(fold) + '.h5')  # creates a HDF5 file 
    with open('model_scores_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(scores, file_pi)
    with open('model_history_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    valid_indexes.append((l, r))
    r += 113350
    l += 113350

with open('model_kfold_indexes', 'wb') as file_pi:
    pickle.dump(valid_indexes, file_pi)   

W0320 16:29:04.418337 12764 deprecation_wrapper.py:119] From C:\Apps\Anaconda3\envs\gpu-cuda10\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



1


W0320 16:29:05.048079 12764 deprecation_wrapper.py:119] From C:\Apps\Anaconda3\envs\gpu-cuda10\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0320 16:29:05.075870 12764 deprecation_wrapper.py:119] From C:\Apps\Anaconda3\envs\gpu-cuda10\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0320 16:29:05.451344 12764 deprecation_wrapper.py:119] From C:\Apps\Anaconda3\envs\gpu-cuda10\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0320 16:29:06.255559 12764 deprecation_wrapper.py:119] From C:\Apps\Anaconda3\envs\gpu-cuda10\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0320 16:29:06.262


Hyper Parameters

Learning Rate: 0.0005
Drop out: 0.3
Batch dim: 64
Number of epochs: 5
Regularizers: 1e-04

Loss: categorical_crossentropy

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputWindow (InputLayer)     (None, 19, 21)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 19, 64)            4096      
_________________________________________________________________
BN1 (BatchNormalization)     (None, 19, 64)            256       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 19, 64)            12352     
_________________________________________________________________
BN2 (BatchNormalization)     (None, 19, 64)            256       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 19, 64)            12352     


W0320 16:29:30.185673 12764 deprecation.py:323] From C:\Apps\Anaconda3\envs\gpu-cuda10\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1020152 samples, validate on 113350 samples
Epoch 1/5
 343360/1020152 [=========>....................] - ETA: 5:52 - loss: 0.8208 - acc: 0.6526 - mean_absolute_error: 0.3027

In [ ]:
kf = KFold(n_splits=10, random_state=10, shuffle=True)
fold = 0
valid_indexes = [([],[])]
for train, test in kf.split(dataset):

    fold += 1
    print(fold)
    
    m = get_model()
    start_time = timer()
    history = m.fit(dataset[train][:,:,0:21], dataset[train][:,:,21 : (21+classSize)], epochs=nn_epochs,
                    batch_size=batch_dim, validation_data=(dataset[test][:,:,0:21], dataset[test][:,:,21 : (21+classSize)]),shuffle=True)
    end_time = timer()
    print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")

    
    scores = m.evaluate(dataset[test][:,:,0:21], dataset[test][:,:,21 : (21+classSize)])
    print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]) + ' fold ' + str(fold))
    
    
    m.save('model_kfold' + str(fold) + '.h5')  # creates a HDF5 file 
    with open('model_scores_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(scores, file_pi)
    with open('model_history_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    valid_indexes.append((train, test))

with open('model_kfold_indexes', 'wb') as file_pi:
    pickle.dump(valid_indexes, file_pi)    
    